In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import re
import os
import warnings

#basic tools 
import os
import warnings

#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys

#metrics 
from sklearn.metrics import roc_curve
import shap
warnings.simplefilter(action='ignore', category=FutureWarning)

Leo todos los datasets menos el de censo ya que da un mejor resultado en el cross validation pero no en el leaderboard público.

In [ ]:
rcc_train = pd.read_csv("/kaggle/input/interbank20/rcc_train.csv")
se_train = pd.read_csv("/kaggle/input/interbank20/se_train.csv", index_col="key_value")
#censo_train = pd.read_csv("/kaggle/input/interbank20/censo_train.csv", index_col="key_value")
sunat_train = pd.read_csv("/kaggle/input/interbank20/sunat_train.csv")
y_train = pd.read_csv("/kaggle/input/interbank20/y_train.csv", index_col="key_value").target

rcc_test= pd.read_csv("/kaggle/input/interbank20/rcc_test.csv")
se_test= pd.read_csv("/kaggle/input/interbank20/se_test.csv", index_col="key_value")
#censo_test= pd.read_csv("/kaggle/input/interbank20/censo_test.csv", index_col="key_value")
sunat_test = pd.read_csv("/kaggle/input/interbank20/sunat_test.csv")

Discretizo la variable condición.

In [ ]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins)
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins)

Creación de la función usada en el benchmark para hacer la cuenta, suma, maximo y minimo de los saldos por cada una de las otras variables:

In [ ]:
def makeCt(df, c, aggfunc=sum):
    try:
        ct = pd.crosstab(df.key_value, df[c].fillna("N/A"), values=df.saldo, aggfunc=aggfunc)
    except:
        ct = pd.crosstab(df.key_value, df[c], values=df.saldo, aggfunc=aggfunc)
    ct.columns = [f"{c}_{aggfunc.__name__}_{v}" for v in ct.columns]
    return ct

Desarrollo de las nuevas variables con la función anterior:

In [ ]:
train = []
test = []
aggfuncs = [len, sum, min, max]
for c in rcc_train.drop(["codmes", "key_value", "saldo"], axis=1):
    print("haciendo", c)
    train.extend([makeCt(rcc_train, c, aggfunc) for aggfunc in aggfuncs])
    test.extend([makeCt(rcc_test, c, aggfunc) for aggfunc in aggfuncs])

In [ ]:
train = pd.concat(train, axis=1)
test = pd.concat(test, axis=1)

Uno los datasets "rcc" con los "se".

In [ ]:
train = train.join(se_train)
test = test.join(se_test)

import gc

del se_train, se_test
gc.collect()

**ANALISIS DATASET SUNAT**



Elimino las lineas que están duplicadas identicamente.

In [ ]:
sunat_train=sunat_train.drop_duplicates()
sunat_test=sunat_test.drop_duplicates()

Discretizo las variables fecalta y fecbaja.

In [ ]:
bins = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5]
sunat_train["fecalta"] = pd.cut(sunat_train.fecalta, bins)
sunat_test["fecalta"] = pd.cut(sunat_test.fecalta, bins)
bins = [-5,-4.5,-4,-3.5,-3,-2.5,-2,-1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5]
sunat_train["fecbaja"] = pd.cut(sunat_train.fecbaja, bins)
sunat_test["fecbaja"] = pd.cut(sunat_test.fecbaja, bins)

Creo una función similar a la creada al principio para "rcc", pero que cuente la cantidad de key_values iguales que hay para cada valor 
de una variable.

In [ ]:
def makeCt_1(df, c, aggfunc=len):
    try:
        ct = pd.crosstab(df.key_value, df[c].fillna("N/A"), values=df.key_value, aggfunc=aggfunc)
    except:
        ct = pd.crosstab(df.key_value, df[c], values=df.key_value, aggfunc=aggfunc)
    ct.columns = [f"{c}_{v}" for v in ct.columns]
    return ct

Creo un ciclo for que aplique la función anterior a cada una de las variables y entrene un LGBM para ver qué variables suman y cuales no.
Hago esto porque Kaggle se queda sin memoria si utilizo todos los atributos directamente.
Excluyo "ubigeo" y "cargorele" porque se queda sin memoria al intentar hacerlo. 

El mismo modelo LGBM sin ninguna de las variables de "sunat" da un auc de 0.8141, por lo tanto me voy a quedar con las variables que, al ser
incluidas en el modelo, den un AUC mayor.

In [ ]:
sunat_train_1 = sunat_train
sunat_test_1 = sunat_test

train_1=train
test_1=test

In [ ]:
import gc
del rcc_train, rcc_test
gc.collect()

In [ ]:
from sklearn.model_selection import ParameterGrid
for c in ['condiciondomicilio', 'estadocontribuyente',
       'codvia', 'codzona', 'contabilidad', 'facturacion', 'domiciliado',
       'comercioexterior', 'codentidadtributo', 'estadotributo','fecalta','fecbaja']:
    print("haciendo", c)
    sunat_train_3 = []
    sunat_test_3 = []
    sunat_train_3.extend([makeCt_1(sunat_train_1, c)])
    sunat_test_3.extend([makeCt_1(sunat_test_1, c)])
    sunat_train_3 = pd.concat(sunat_train_3, axis=1)
    sunat_test_3 = pd.concat(sunat_test_3, axis=1)
    test = pd.merge(test_1,sunat_test_3, on= 'key_value' ,how='left')
    del sunat_test_3
    gc.collect()
    train = pd.merge(train_1,sunat_train_3, on= 'key_value' ,how='left')
    del sunat_train_3
    gc.collect()
    keep_cols = list(set(train.columns).intersection(set(test.columns)))
    train = train[keep_cols]
    test = test[keep_cols]
    len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))
    test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
    train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
    folds = [train.index[t] for t, v in KFold(5).split(train)]
    params = ParameterGrid({"min_child_samples": [1000], "boosting_type": ["gbdt"]})
    best_score = 0
    best_probs = []
    for param in params:
        test_probs = []
        train_probs = []
        p  = "///".join([f"{k}={v}" for k, v in param.items()])
        print("*"*10, p, "*"*10)
        for i, idx in enumerate(folds):
            Xt = train.loc[idx]
            yt = y_train.loc[Xt.index]

            Xv = train.drop(Xt.index)
            yv = y_train.loc[Xv.index]

            learner = LGBMClassifier(n_estimators=1000, **param)
            learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                        eval_set=[(Xt, yt), (Xv, yv)], verbose=False)
            test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
            train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))

        test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
        train_probs = pd.concat(train_probs)
        score = roc_auc_score(y_train, train_probs.loc[y_train.index])
        print(f"roc auc estimado para {p} con {c}: {score}")
        if score > best_score:
            print("*"*10, f"{p} es el nuevo mejor modelo", "*"*10)
            best_score = score
            best_probs = test_probs


En el ciclo anterior veo que todas agregan valor excepto por "codvia". Por lo tanto las agrego como categoricas, contando la cantidad de key_values por cada valor de variable.

In [ ]:
sunat_train_2 = []
sunat_test_2 = []
for c in ["tipcontribuyente","tippersona","ciiu","condiciondomicilio","estadocontribuyente","codzona","contabilidad","facturacion","domiciliado","comercioexterior","codentidadtributo","estadotributo","fecalta","fecbaja"]:
    print("haciendo", c)
    sunat_train_2.extend([makeCt_1(sunat_train, c)])
    sunat_test_2.extend([makeCt_1(sunat_test, c)])

In [ ]:
sunat_train_2 = pd.concat(sunat_train_2, axis=1)
sunat_test_2 = pd.concat(sunat_test_2, axis=1)

Uno los datasets train y test con las nuevas variables creadas de sunat:

In [ ]:
train = pd.merge(train_1,sunat_train_2, on= 'key_value' ,how='left')
test = pd.merge(test_1,sunat_test_2, on= 'key_value' ,how='left')

In [ ]:
del sunat_train_2
del sunat_test_2
del test_1
del train_1
gc.collect()

In [ ]:
test.to_csv('test_complete.csv')
train.to_csv('train_complete.csv')

***************************************************************************************************************************************

**MODELO**: Continúa en el Script FINAL_2